## Описание задачи:

Будем решать задачу идентификации взломщика по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в статье на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".

В этом соревновании будем решать похожую задачу: алгоритм будет анализировать последовательность из нескольких веб-сайтов, посещенных подряд одним и тем же человеком, и определять, Элис это или взломщик (кто-то другой).

Данные собраны с прокси-серверов Университета Блеза Паскаля. "A Tool for Classification of Sequential Data", авторы Giacomo Kahn, Yannick Loiseau и Olivier Raynaud.

----

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import vstack, hstack, csc_matrix
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit,\
                                    cross_val_score, cross_validate, GridSearchCV, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from collections import defaultdict
import time
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce

In [ ]:
class FeatureSelector():
    def __init__(self, estimator,
                       metric,\
                       larger_is_better,\
                       cv, 
                       use_values,
                       use_recursion,
                       fill_na,\
                       show_progress, 
                       early_stopping = None):
        '''
        Инициализирует модель для отбора признаков
        
        Параметры:
            1) estimator - модель
            2) metric - метрика качества (названия метрик sklearn + может быть кастомная)
            3) larger_is_better - критерий оптимизации (чем больше, тем лучше)
            4) cv - схема валидации
            5) use_values - индексы столбцов, в которых требуется отобрать значения
            6) use_recursion - использовать рекурсию в отборе
            7) fill_na - значение, которым заполняются np.nan
            8) show_progress - печатать результаты валидации
            9) early_stopping - число итераций без улучшения метрики для ранней остановки отбора
        Возвращает:
            1) fit - производит отбор признаков
            2) transform - оставляет отобранные признаки
            3) return_self - возвращает 
                - best_features - отобранные признаки(список)
                - D_best_features - отобранные значения признаков (словарь: {признак:значения})
                - best_score - лучшее значение метрики
        '''
        self.estimator = estimator
        self.metric = metric
        self.cv = cv
        self.use_values = use_values        
        self.use_recursion = use_recursion
        self.show_progress = show_progress
        self.early_stopping = early_stopping
        self.fill_na = fill_na
        self.larger_is_better = larger_is_better
    def fit(self, X, Y):        
        flag = isinstance(X[:, 0], csc_matrix)
        # список с результатами валидации
        column_value_score = []
        # проходим по признакам
        for i in tqdm_notebook(range(X.shape[1])):
            # если формат матрицы признаков == csc_matrix
            if flag:
                # выбираем столбец, преобразуем
                ser = pd.DataFrame(X[:, i].todense())[0].values.flatten()
            # если формат != csc_matrix
            else:
                # выбираем столбец
                ser = X[:, i]        
            # если столбец в списке с проверкой значений 
            if self.use_values is not None:                
                if i in self.use_values:
                    # уникальные значения столбца
                    unique_values = np.unique(ser)  
                    # валидируем каждое значение
                    for val in unique_values:
                        _x = np.int32(ser==val).reshape(-1,1)
                        column_value_score.append((i, val,\
                                                   cross_val_score(self.estimator,\
                                                                   _x, Y,\
                                                                   scoring = self.metric,\
                                                                   cv = self.cv).mean()))
                else: 
                    # валидируем столбец
                    column_value_score.append((i, None,\
                                               cross_val_score(self.estimator,\
                                                               _x, Y,\
                                                               scoring = self.metric,\
                                                               cv = self.cv).mean()))
            else:
                # валидируем столбец
                    column_value_score.append((i, None,\
                                               cross_val_score(self.estimator,\
                                                               ser.reshape(-1,1), Y,\
                                                               scoring = self.metric,\
                                                               cv = self.cv).mean()))
                

        # признаки и значения признаков в порядке убывания валидации
        order = np.array(sorted(column_value_score, key = lambda x: x[-1], reverse = True))[:, :2]             
        # список лучших признаков
        best_features = []
        # словарь лучших значений признаков
        D_best_features = defaultdict(list)
        # список с признаками, не давшими прироста
        to_drop = []
        
        # лучшее значение метрики
        if self.larger_is_better:
            best_score = 0
        else:
            best_score = np.inf            
        counter = 0
        # проходим по признакам и значениям признаков в порядке убывания валидации
        for feature, feature_value in tqdm_notebook(order):   

            # добавляем текущие признаки/значения
            if feature_value is None:
                best_features.append(feature)               
            else:
                D_best_features[feature].append(feature_value)

            # обновляем матрицы
            L = []
            for k, v in D_best_features.items():
                if isinstance(X[:, k], csc_matrix):
                    L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))
                else:
                    L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))

            if flag:
                if (len(best_features)>0) & (len(L)>0):
                    _X = csc_matrix(hstack([X[:, best_features], csc_matrix(np.column_stack(L)) ]))
                elif (len(best_features)==0) & (len(L)>0):
                    _X = csc_matrix(np.column_stack(L))
                elif (len(best_features)>0) & (len(L)==0):
                    _X = csc_matrix(X[:, best_features])                    
                        
            else:
                if (len(best_features)>0) & (len(L)>0):
                    _X = np.column_stack([X[:, best_features], np.column_stack(L)])
                elif (len(best_features)==0) & (len(L)>0):
                    _X = np.column_stack(L)
                elif (len(best_features)>0) & (len(L)==0):
                    _X = X[:, best_features] 
            # считаем валидацию    
            current_score = cross_val_score(self.estimator, _X, Y, scoring = self.metric, cv = self.cv).mean()
            # если метрика улучшилась
            if self.larger_is_better:
                if current_score>best_score:
                    # обновляем лучшую метрику
                    best_score = current_score
                    counter = 0
                    # печатаем 
                    if self.show_progress:
                        print('new best_score = {}'.format(best_score))
                # если метрика не улучшилась
                else: 
                    counter+=1
                    # удаляем признак/значение
                    if feature_value is None:
                        best_features = [val for val in best_features if val != feature]
                        to_drop.append((feature, None))
                    else:
                        D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                        to_drop.append((feature, feature_value))
                    if counter == self.early_stopping:
                        break
            else:
                if current_score<best_score:
                    # обновляем лучшую метрику
                    best_score = current_score
                    counter = 0
                    # печатаем 
                    if self.show_progress:
                        print('new best_score = {}'.format(best_score))
                    # если метрика не улучшилась
                else: 
                    counter+=1
                    # удаляем признак/значение
                    if feature_value is None:
                        best_features = [val for val in best_features if val != feature]
                        to_drop.append((feature, None))
                    else:
                        D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                        to_drop.append((feature, feature_value))
                    if counter == self.early_stopping:
                        break

        if self.use_recursion:
            # запускаем бесконечный цикл
            while True:
                # списки лучших признаков до и после
                to_drop_before = to_drop
                to_drop_after = []
                # проходим по признакам и значениям признаков в порядке убывания валидации
                for feature, feature_value in tqdm_notebook(to_drop_before):   
                    # добавляем текущие признаки/значения
                    if feature_value is None:
                        best_features.append(feature)               
                    else:
                        D_best_features[feature].append(feature_value)

                    # обновляем матрицы
                    L = []
                    for k, v in D_best_features.items():
                        if isinstance(X[:, k], csc_matrix):
                            L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))
                        else:
                            L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))

                    if flag:
                        if (len(best_features)>0) & (len(L)>0):
                            _X = csc_matrix(hstack([X[:, best_features], csc_matrix(np.column_stack(L)) ]))
                        elif (len(best_features)==0) & (len(L)>0):
                            _X = csc_matrix(np.column_stack(L))
                        elif (len(best_features)>0) & (len(L)==0):
                            _X = csc_matrix(X[:, best_features])                    

                    else:
                        if (len(best_features)>0) & (len(L)>0):
                            _X = np.column_stack([X[:, best_features], np.column_stack(L)])
                        elif (len(best_features)==0) & (len(L)>0):
                            _X = np.column_stack(L)
                        elif (len(best_features)>0) & (len(L)==0):
                            _X = X[:, best_features] 

                    # считаем валидацию    
                    current_score = cross_val_score(self.estimator, _X, Y, scoring = self.metric, cv = self.cv).mean()
                    
                    if self.larger_is_better:
                        if current_score>best_score:
                            # обновляем лучшую метрику
                            best_score = current_score
                            counter = 0
                            # печатаем 
                            if self.show_progress:
                                print('new best_score = {}'.format(best_score))
                            # если метрика не улучшилась
                        else: 
                            # удаляем признак/значение
                            if feature_value is None:
                                best_features = [val for val in best_features if val != feature]
                                to_drop_after.append((feature, None))
                            else:
                                D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                                to_drop_after.append((feature, feature_value))
                    else:
                        if current_score<best_score:
                            # обновляем лучшую метрику
                            best_score = current_score
                            counter = 0
                            # печатаем 
                            if self.show_progress:
                                print('new best_score = {}'.format(best_score))
                        else: 
                            # удаляем признак/значение
                            if feature_value is None:
                                best_features = [val for val in best_features if val != feature]
                                to_drop_after.append((feature, None))
                            else:
                                D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                                to_drop_after.append((feature, feature_value))
                    

                # если списки одинаковые, останавливаем отбор
                if len(to_drop_after) == len(to_drop_before):
                    break
                # если разные - обновляем списки до и после
                else:
                    to_drop_before = to_drop_after
                    to_drop_after = []
                    
        self.best_features = best_features
        self.D_best_features = D_best_features
        self.best_score =best_score
        self.flag = flag
    def transform(self, X):
              
        if len(self.best_features) !=0:
            x1 = X[:, self.best_features]
        else:
            x1 = None
        if len(list(self.D_best_features.keys())) !=0:
            L=[]
            for k, v in self.D_best_features.items():
                if self.flag:
                    L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))                    
                else:
                    L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))
            x2 = np.column_stack(L)
        else:
            x2 = None
            
        if (x1 is not None) & (x2 is not None):
            if self.flag: 
                _X = csc_matrix(hstack([x1, x2]))
            else:
                _X = np.column_stack([x1, x2])
                
        if (x1 is not None) & (x2 is None):
            _X = x1
        if (x1 is None) & (x2 is not None):
            if self.flag:
                _X = csc_matrix(x2)
            else:
                _X = x2
        return _X     
        
    def return_self(self):
        return self
    
def _get_tfidf_and_time_features(df_tr, df_te, target_col, vectorizer):
    '''
    параметры:
        1) df_tr - тренировочный датасет
        2) df_te - тестовый датасет
        3) target_col - колонка с целевой переменной (df_tr)
    действия:
        1) трансформирует сайты с помощью tfidf
        2) извлекает признаки из даты
    вывод:
        1) _X_logit_tr, _X_logit_te - признаки для логистической регрессии
        2) _X_lgb_tr, _X_lgb_te - признаки для бустинга
        3) _y_tr - целевая переменная
        4) to_check_values_idxs - индексы признаков, где необходимо отобрать значения
    '''
    
    # уникальные сайты в тест части
    unique_sites_te = pd.Series(df_te[SITES].values.flatten()).dropna().sort_values().unique()

    # уникальные сайты, посещенные Элис
    unique_sites_alice = pd.Series(df_tr[df_tr[target_col]==1][SITES].values.flatten()).dropna().sort_values().unique()

    # сайты, которые оставляем
    sites_to_use = np.intersect1d(unique_sites_te, unique_sites_alice)

    # оставляем сайты, посещенные в тест части & посещенные Элис
    sites_tr = df_tr[SITES].applymap(lambda x: x if x in sites_to_use else FILL_NA)
    sites_te = df_te[SITES].applymap(lambda x: x if x in sites_to_use else FILL_NA)

    # начало сессии 
    start_tr = df_tr[TIMES].min(1)
    start_te = df_te[TIMES].min(1)

    ts_columns = ['dayofyear', 'weekofyear', 'year', 'month',\
                  'day', 'dayofweek', 'hour']

    D_ts = {}
    # извлекаем признаки
    for key, ts_df in zip(('tr', 'te'), (start_tr, start_te)):    
        times_df = pd.DataFrame(np.column_stack([ts_df.dt.dayofyear,\
                             ts_df.dt.weekofyear,\
                             ts_df.dt.year,\
                             ts_df.dt.month,\
                             ts_df.dt.day,\
                             ts_df.dt.dayofweek,\
                             ts_df.dt.hour]))
        times_df.columns = ts_columns
        D_ts[key] = times_df

    # оставляем значения из теста
    for col in D_ts['tr'].columns:
        values_to_use = np.unique(D_ts['te'][col])
        D_ts['tr'][col] = D_ts['tr'][col].apply(lambda x: x if x in values_to_use else FILL_NA)    

    # one hot encoding
    ohe_encoder = OneHotEncoder(handle_unknown='ignore')
    ohe_encoder.fit(D_ts['tr'])

    ts_ohe_tr = ohe_encoder.transform(D_ts['tr'])
    ts_ohe_te = ohe_encoder.transform(D_ts['te'])

    # готовим данные для tfidf
    sites_full_str = pd.concat([sites_tr, sites_te], 0)\
                       .astype(int).astype(str)\
                       .apply(lambda row: '_'.join(row), axis =1)
    
    # продолжительность сессии (в секундах+log)
    duration_log_tr = np.log1p((df_tr[TIMES].max(1) - df_tr[TIMES].min(1)) / np.timedelta64(1, 's'))
    duration_log_te = np.log1p((df_te[TIMES].max(1) - df_te[TIMES].min(1)) / np.timedelta64(1, 's'))

    # трансформируем сайты    
    tfidf_full = csc_matrix(vectorizer.fit_transform(sites_full_str))
    tfidf_tr, tfidf_te = tfidf_full[:len(df_tr)], tfidf_full[len(df_tr):]
    # датасеты для лоигта
    _X_logit_tr = csc_matrix(hstack([ts_ohe_tr, tfidf_tr, duration_log_tr.values.reshape(-1, 1)]))
    _X_logit_te = csc_matrix(hstack([ts_ohe_te, tfidf_te, duration_log_te.values.reshape(-1, 1)]))        
    # датасеты для бустинга
    _X_lgb_tr = csc_matrix(hstack([D_ts['tr'], tfidf_tr, duration_log_tr.values.reshape(-1, 1)]))
    _X_lgb_te = csc_matrix(hstack([D_ts['te'], tfidf_te, duration_log_te.values.reshape(-1, 1)]))
    # таргет
    _y_tr = np.array(y_tr)        
    # индексы признаков, в которых надо тотбрать значения
    to_check_values_idxs = np.arange(D_ts['tr'].shape[1]).tolist()
    
    return {'_X_logit_tr':_X_logit_tr, '_X_logit_te':_X_logit_te,\
            '_X_lgb_tr':_X_lgb_tr, '_X_lgb_te':_X_lgb_te,\
            '_y_tr':_y_tr, 'to_check_values_idxs':to_check_values_idxs}

def _find_optimal_folding(y, cvs):    
    # максимальные различия по фолдам
    max_diffs_by_folds_L = []    
    # итерируемся по числу фолдов
    for cv in cvs:
        # список с l1 нормами для среднего таргета на трейн и валидационном фолдах
        l1_diff_L = []
        for tr_idx, val_idx in cv.split(y):
            l1_diff_L.append(np.abs(df_tr.iloc[tr_idx]['target'].mean() - df_tr.iloc[val_idx]['target'].mean()))
        # максимальная l1 за всю валидацию
        max_diffs_by_folds_L.append(np.max(l1_diff_L))
    # минимизируем максимальный разброс --> получаем число фолдов
    return cvs[np.argmin(max_diffs_by_folds_L)]

class StackingEstimator():    
    def __init__(self, models, cv, subsample, seed):
        '''
        models - список с ансамблем моделей
        cvs - список с схемами валидации
        n_iterations - итераций стекинга
        subsamples - список с долями используемых признаков 
        seed - генератор случайных чисел
        '''        
        self.models = models
        self.cv = cv        
        self.subsamples = subsamples
        self.seed = seed
    def fit(self, X, y):
        '''
        1) обучаем модели на валидации
        2) сохраняем обученные модели
        '''        
        nfeat = X.shape[1]
        feat_idxs = np.arange(nfeat)        
        estimators_use_idxs=[]
        for model_idx, model in tqdm_notebook(enumerate(self.models), total = len(self.models)):
            np.random.seed(model_idx+self.seed+1)
            use_idxs = np.random.choice(feat_idxs, np.int32(np.around(nfeat*subsample)), replace = False)
            for tr_idx, val_idx in cv.split(y):
                model.fit(X[tr_idx][:, use_idxs], y[tr_idx])   
                estimators_use_idxs.append((model, use_idxs))        
        self.estimators_use_idxs = estimators_use_idxs
    def get_metafeatures(self, X):
        '''
        с помощью обученных моделей получаем метапризнаки
        '''
        L = []    
        for use_idxs, estimator in self.estimators_use_idxs:
            try:
                L.append(estimator.predict_proba(X[:, use_idxs])[:, 1])
            except:
                L.append(estimator.predict(X[:, use_idxs]))
                
        return np.column_stack(L)

<img src="pipeline.png">

In [3]:
# сайты, время
TIMES = ['time%d' % i for i in range(1, 11)]
SITES = ['site%d' % i for i in range(1, 11)]

# трейн, тест
df_tr = pd.read_csv('train_sessions.csv', parse_dates = TIMES).set_index('session_id').sort_values('time1')
df_te = pd.read_csv('test_sessions.csv', parse_dates = TIMES).set_index('session_id')

# таргет
y_tr = df_tr['target']

In [4]:
# константы
SEED = 13
FILL_NA = -1
TRAIN_SHARE = .9
IDX_SPLIT = np.int32(np.around(TRAIN_SHARE*len(df_tr)))
NFOLDS_L = np.arange(3, 13)

# модели
lgb_clf = LGBMClassifier(random_state = SEED)
logit_clf = LogisticRegression(random_state = SEED)

In [5]:
# находим число фолдов, при котором абсолютный разброс среднего таргета в трейне и отложенной частях минимален
# --> тестируем модель примерно на том, на чем обучали
L_cvs = [TimeSeriesSplit(k) for k in range(3, 11)]
tscv = _find_optimal_folding(df_tr['target'].iloc[:IDX_SPLIT], L_cvs)

In [6]:
tfidf_param_grid = {'ngram_range':((1, 1), (1, 2), (1, 3), (1, 4), (1, 5)),\
                    'max_features':(10, 100, 200)}                 

#### перебор параметров tf-idf

In [7]:
L_scores = []
L_XX_logit, L_XX_lgb = [], []
# инициализируем tfidf
for ngram_range in tqdm_notebook(tfidf_param_grid['ngram_range']):
    for max_features in tqdm_notebook(tfidf_param_grid['max_features']):
        
        # получаем признаки
        D_features = _get_tfidf_and_time_features(df_tr, df_te,\
                                                  target_col = 'target',\
                                                  vectorizer = TfidfVectorizer(ngram_range = ngram_range,\
                                                                               max_features = max_features)) 
        
        # отбираем признаки бустингом
        lgb_selector = FeatureSelector(estimator = lgb_clf,\
                                                   metric = 'roc_auc',\
                                                   larger_is_better = True,\
                                                   cv = tscv,\
                                                   use_values = D_features['to_check_values_idxs'],\
                                                   use_recursion = False,\
                                                   fill_na = FILL_NA,\
                                                   show_progress = False, 
                                                   early_stopping = None)            
        lgb_selector.fit(D_features['_X_lgb_tr'][:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])

        # auc на валидации
        auc_lgb_cv = lgb_selector.return_self().best_score

        # отобранные признаки
        _X_lgb_sel_tr = lgb_selector.transform(D_features['_X_lgb_tr'])
        _X_lgb_sel_te = lgb_selector.transform(D_features['_X_lgb_te'])

        # auc на отложенной
        lgb_clf.fit(_X_lgb_sel_tr[:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])
        auc_lgb_hold = roc_auc_score(D_features['_y_tr'][IDX_SPLIT:],\
                                         lgb_clf.predict_proba(_X_lgb_sel_tr[IDX_SPLIT:])[:, 1])
            
        # отбираем признаки бустингом
        logit_selector = FeatureSelector(estimator = logit_clf,\
                                                   metric = 'roc_auc',\
                                                   larger_is_better = True,\
                                                   cv = tscv,\
                                                   use_values = None,\
                                                   use_recursion = False,\
                                                   fill_na = FILL_NA,\
                                                   show_progress = False, 
                                                   early_stopping = None)            
        logit_selector.fit(D_features['_X_logit_tr'][:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])

        # auc на валидации
        auc_logit_cv = logit_selector.return_self().best_score

        # отобранные признаки
        _X_logit_sel_tr = logit_selector.transform(D_features['_X_logit_tr'])
        _X_logit_sel_te = logit_selector.transform(D_features['_X_logit_te'])

        # auc на отложенной
        logit_clf.fit(_X_logit_sel_tr[:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])
        auc_logit_hold = roc_auc_score(D_features['_y_tr'][IDX_SPLIT:],\
                                           logit_clf.predict_proba(_X_logit_sel_tr[IDX_SPLIT:])[:, 1]) 
        
        L_scores.append((ngram_range, max_features, auc_lgb_cv, auc_lgb_hold, auc_logit_cv, auc_logit_hold))
        L_XX_logit.append((_X_logit_sel_tr, _X_logit_sel_te))
        L_XX_lgb.append((_X_lgb_sel_tr, _X_lgb_sel_te))

In [8]:
# with open ('L_scores.pickle', 'wb') as f:
#     pickle.dump(L_scores, f)

In [24]:
cvAB = pd.DataFrame(L_scores, columns = ['ngram_range', 'max_features', 'lgb_cv', 'lgb_hold', 'logit_cv', 'logit_hold'])

In [25]:
best_idx = np.argmax(np.max(np.column_stack([cvAB[['lgb_cv', 'lgb_hold']].mean(1),\
                                                cvAB[['logit_cv', 'logit_hold']].mean(1)]), 1))

In [26]:
cvAB.iloc[best_idx].to_frame('best_params').T

,ngram_range,max_features,lgb_cv,lgb_hold,logit_cv,logit_hold
best_params,"(1, 3)",100,0.954424,0.830041,0.962221,0.95935


#### max_features окрестность №1

In [39]:
L2_scores, L2_XX_logit = [], []
# инициализируем tfidf
for max_features in tqdm_notebook((90, 100, 110)):
    # получаем признаки
    D_features = _get_tfidf_and_time_features(df_tr, df_te,\
                                                  target_col = 'target',\
                                                  vectorizer = TfidfVectorizer(ngram_range = (1, 3),\
                                                                               max_features = max_features)) 
    # отбираем признаки бустингом
    logit_selector = FeatureSelector(estimator = logit_clf,\
                                                   metric = 'roc_auc',\
                                                   larger_is_better = True,\
                                                   cv = tscv,\
                                                   use_values = None,\
                                                   use_recursion = False,\
                                                   fill_na = FILL_NA,\
                                                   show_progress = False, 
                                                   early_stopping = None)            
    logit_selector.fit(D_features['_X_logit_tr'][:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])

    # auc на валидации
    auc_logit_cv = logit_selector.return_self().best_score

    # отобранные признаки
    _X_logit_sel_tr = logit_selector.transform(D_features['_X_logit_tr'])
    _X_logit_sel_te = logit_selector.transform(D_features['_X_logit_te'])

    # auc на отложенной
    logit_clf.fit(_X_logit_sel_tr[:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])
    auc_logit_hold = roc_auc_score(D_features['_y_tr'][IDX_SPLIT:],\
                                           logit_clf.predict_proba(_X_logit_sel_tr[IDX_SPLIT:])[:, 1]) 
    
    L2_scores.append((max_features, auc_logit_cv, auc_logit_hold))
    L2_XX_logit.append((_X_logit_sel_tr, _X_logit_sel_te))
    

In [45]:
mean_scores = pd.DataFrame(L2_scores).set_index(0)[[1, 2]].mean(1)
best_score = mean_scores.max()
best_max_features = mean_scores.idxmax()

#### max_features окрестность №2

In [55]:
low, mid, high = 89, 90, 91
for max_features in tqdm_notebook((low, mid, high)):     
    # получаем признаки
    D_features = _get_tfidf_and_time_features(df_tr, df_te,\
                                                  target_col = 'target',\
                                                  vectorizer = TfidfVectorizer(ngram_range = (1, 3),\
                                                                               max_features = max_features)) 
    # отбираем признаки бустингом
    logit_selector = FeatureSelector(estimator = logit_clf,\
                                                   metric = 'roc_auc',\
                                                   larger_is_better = True,\
                                                   cv = tscv,\
                                                   use_values = None,\
                                                   use_recursion = False,\
                                                   fill_na = FILL_NA,\
                                                   show_progress = False, 
                                                   early_stopping = None)            
    logit_selector.fit(D_features['_X_logit_tr'][:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])

    # auc на валидации
    auc_logit_cv = logit_selector.return_self().best_score

    # отобранные признаки
    _X_logit_sel_tr = logit_selector.transform(D_features['_X_logit_tr'])
    _X_logit_sel_te = logit_selector.transform(D_features['_X_logit_te'])

    # auc на отложенной
    logit_clf.fit(_X_logit_sel_tr[:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])
    auc_logit_hold = roc_auc_score(D_features['_y_tr'][IDX_SPLIT:],\
                                           logit_clf.predict_proba(_X_logit_sel_tr[IDX_SPLIT:])[:, 1]) 
    
    # средний auc на валидации, на отложенной выборке
    mean_score = (auc_logit_cv+auc_logit_hold)/2
    
    # если улучшение
    if mean_score>best_score:
        best_score = mean_score
        best_max_features = new_max_features    
        
if best_max_features == mid:
    print('best_max_features = 90')
else:
    if best_max_features == high:
        while True:
            new_max_features = best_max_features+1
            # получаем признаки
            D_features = _get_tfidf_and_time_features(df_tr, df_te,\
                                                          target_col = 'target',\
                                                          vectorizer = TfidfVectorizer(ngram_range = (1, 3),\
                                                                                       max_features = new_max_features)) 
            # отбираем признаки бустингом
            logit_selector = FeatureSelector(estimator = logit_clf,\
                                                           metric = 'roc_auc',\
                                                           larger_is_better = True,\
                                                           cv = tscv,\
                                                           use_values = None,\
                                                           use_recursion = False,\
                                                           fill_na = FILL_NA,\
                                                           show_progress = False, 
                                                           early_stopping = None)            
            logit_selector.fit(D_features['_X_logit_tr'][:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])

            # auc на валидации
            auc_logit_cv = logit_selector.return_self().best_score

            # отобранные признаки
            _X_logit_sel_tr = logit_selector.transform(D_features['_X_logit_tr'])
            _X_logit_sel_te = logit_selector.transform(D_features['_X_logit_te'])

            # auc на отложенной
            logit_clf.fit(_X_logit_sel_tr[:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])
            auc_logit_hold = roc_auc_score(D_features['_y_tr'][IDX_SPLIT:],\
                                                   logit_clf.predict_proba(_X_logit_sel_tr[IDX_SPLIT:])[:, 1]) 

            # средний auc на валидации, на отложенной выборке
            mean_score = (auc_logit_cv+auc_logit_hold)/2

            # если улучшение
            if mean_score>best_score:
                best_score = mean_score
                best_max_features = new_max_features  
            else:
                print('best_max_features = {}'.format(best_max_features))    
                break
    else:
        while True:
            new_max_features = best_max_features-1
            # получаем признаки
            D_features = _get_tfidf_and_time_features(df_tr, df_te,\
                                                          target_col = 'target',\
                                                          vectorizer = TfidfVectorizer(ngram_range = (1, 3),\
                                                                                       max_features = new_max_features)) 
            # отбираем признаки бустингом
            logit_selector = FeatureSelector(estimator = logit_clf,\
                                                           metric = 'roc_auc',\
                                                           larger_is_better = True,\
                                                           cv = tscv,\
                                                           use_values = None,\
                                                           use_recursion = False,\
                                                           fill_na = FILL_NA,\
                                                           show_progress = False, 
                                                           early_stopping = None)            
            logit_selector.fit(D_features['_X_logit_tr'][:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])

            # auc на валидации
            auc_logit_cv = logit_selector.return_self().best_score

            # отобранные признаки
            _X_logit_sel_tr = logit_selector.transform(D_features['_X_logit_tr'])
            _X_logit_sel_te = logit_selector.transform(D_features['_X_logit_te'])

            # auc на отложенной
            logit_clf.fit(_X_logit_sel_tr[:IDX_SPLIT], D_features['_y_tr'][:IDX_SPLIT])
            auc_logit_hold = roc_auc_score(D_features['_y_tr'][IDX_SPLIT:],\
                                                   logit_clf.predict_proba(_X_logit_sel_tr[IDX_SPLIT:])[:, 1]) 

            # средний auc на валидации, на отложенной выборке
            mean_score = (auc_logit_cv+auc_logit_hold)/2

            # если улучшение
            if mean_score>best_score:
                best_score = mean_score
                best_max_features = new_max_features
            else:
                print('best_max_features = {}'.format(best_max_features))                
                break
    

best_max_features = 90


#### используем весь трейн(мы выбрали модель, которая не переобучается на трейн датасет)

In [ ]:
D_features = _get_tfidf_and_time_features(df_tr, df_te,\
                                          target_col = 'target',\
                                          vectorizer = TfidfVectorizer(ngram_range = (1, 3),\
                                                                       max_features = 90)) 
# отбираем признаки на всем трейне, используем рекурсию
logit_selector = FeatureSelector(estimator = logit_clf,\
                                 metric = 'roc_auc',\
                                 larger_is_better = True,\
                                 cv = tscv,\
                                 use_values = None,\
                                 use_recursion = True,\
                                 fill_na = FILL_NA,\
                                 show_progress = True, 
                                 early_stopping = None)            
logit_selector.fit(D_features['_X_logit_tr'], D_features['_y_tr'])
_X_logit_sel_tr = logit_selector.transform(D_features['_X_logit_tr'])
_X_logit_sel_te = logit_selector.transform(D_features['_X_logit_te'])

new best_score = 0.7996124543428651
new best_score = 0.8092685831803784
new best_score = 0.813276062815821
new best_score = 0.8452663797945312
new best_score = 0.8570511312789667
new best_score = 0.8947537912504863
new best_score = 0.9018019420630043
new best_score = 0.9097443848215407
new best_score = 0.9170488334786365
new best_score = 0.9226383193097389
new best_score = 0.926896101683679
new best_score = 0.9323238559254446
new best_score = 0.936469739825848
new best_score = 0.941293599914627
new best_score = 0.9466248632728534
new best_score = 0.9474764694171118
new best_score = 0.9479286823158859
new best_score = 0.9481615376413506
new best_score = 0.9487822619253508
new best_score = 0.9495739587714154
new best_score = 0.951906354403606
new best_score = 0.9529583193788904
new best_score = 0.9530077709803575
new best_score = 0.9530412987496282
new best_score = 0.9535215407636237
new best_score = 0.9538852070339122
new best_score = 0.9540997547566381
new best_score = 0.95411511394210

In [ ]:
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

# базовые модели
models_L = [LogisticRegression(random_state = SEED),\
            Lasso(random_state = SEED),\
            Ridge(random_state = SEED),\
            DecisionTreeRegressor(random_state = SEED),\
            RandomForestRegressor(n_estimators = 20, random_state = SEED),\
            KNeighborsRegressor()]

_y_tr = D_features['_y_tr']
_X_logit_sel_tr = _X_logit_sel_tr.todense()
_X_logit_sel_te = _X_logit_sel_te.todense()

In [ ]:
stack1_reg = StackingEstimator(models = models_L,\ # базовые модели                              
                              cv = KFold(20), \ # схемы валидации для получения метапризнаков
                              subsample = .8,\ # доли подпростарнств используемых признаков
                              seed = SEED) # генератор случайных чисел

In [ ]:
stack1_reg.fit(_X_logit_sel_tr[:IDX_SPLIT], _y_tr[:IDX_SPLIT])
X_meta_tr = stack_reg.transform(_X_logit_sel_tr)
X_meta_te = stack_reg.transform(_X_logit_sel_te)

In [ ]:
logit_selector = FeatureSelector(estimator = logit_clf,\
                                 metric = 'roc_auc',\
                                 larger_is_better = True,\
                                 cv = tscv,\
                                 use_values = None,\
                                 use_recursion = False,\
                                 fill_na = FILL_NA,\
                                 show_progress = False, 
                                 early_stopping = None)            
logit_selector.fit(X_meta_tr[:IDX_SPLIT], _y_tr[:IDX_SPLIT])
_X_logit_sel_tr = logit_selector.transform(X_meta_tr)
_X_logit_sel_te = logit_selector.transform(X_meta_te)

In [ ]:
best_score  = logit_selector.return_self().best_score

In [ ]:
# вес1
for w_logit in np.linspace(0, 1, 100):
    
    # вес2
    w_lgb = 1-w_logit
    
    scores = []
    for tr_idx, val_idx in tscv.split(_y_tr[:IDX_SPLIT]):
        
        # трейн, валидация 
        xtr, xval = _X_logit_sel_tr[tr_idx], _X_logit_sel_tr[val_idx]
        ytr, yval = _y_tr[tr_idx], _y_tr[val_idx]
        
        # фитим модели
        logit_clf.fit(xtr, ytr)
        lgb_clf.fit(xtr, ytr)
        
        # делаем предсказания
        predprob_logit = logit_clf.predict_proba(xval)[:, 1]
        predprob_lgb = lgb_clf.predict_proba(xval)[:, 1]
        
        # смешиваем
        y_blend = predprob_logit*w_logit+predprob_lgb*w_lgb
        
        scores.append(roc_auc_score(yval, y_blend))
        
    current_score = np.mean(scores)        
    if current_score > best_score:
        best_score = current_score
        best_w_logit = w_logit
        best_w_lgb = 1-best_w_logit
        

In [ ]:
print('w_logit = {}, w_lgb = {}'.format(best_w_logit, best_w_lgb))

In [ ]:
logit_clf.fit(_X_logit_sel_tr, _y_tr)
lgb_clf.fit(_X_logit_sel_tr, _y_tr)
y_predprob_logit = logit_clf.predict_proba(_X_logit_sel_te)
y_predprob_lgb = lgb_clf.predict_proba(_X_logit_sel_te)

y_blend_te = y_predprob_logit*best_w_logit + y_predprob_lgb*best_w_lgb